In [ ]:
from spider_lianjia.spiders.chengjiao import get_chengjiao_one
from spyder_chengjiao import insert_db
g1=get_chengjiao_one().get_chengjiao('第五郡四号地')
list=[]
for i in range(0,len(g1)):
    list.append({'title':g1[i][0],'room':g1[i][3],'area':g1[i][5],'dealDate':g1[i][11],'totalPrice':g1[i][1],'unitPrice':g1[i][2],'dealCycle':g1[i][10],
    'guaPai':g1[i][9],'seller':0,'district':g1[i][7],'quyu':g1[i][6]})
insert_db(list)

In [ ]:
#对成交进行数据处理
from pandas import DataFrame,Series
import pandas as pd
import re
df_c=DataFrame(g1,columns=['title','totalPrice','unitPrice','room','type','area','quxian','district','builtYear','guapaiPrice','dealCycle','dealDate'])
df_c['area']=df_c['area'].apply(lambda x:float(re.search('\d+(\.\d+)?',x).group()))
df_c=df_c.astype({'totalPrice':'float','unitPrice':'float','guapaiPrice':'float','dealCycle':'int'})
df_c['builtYear']=df_c['builtYear'].apply(lambda x:int(x) if x!='未知 ' else 0)
df_c['dealDate']=pd.to_datetime(df_c['dealDate'])

In [ ]:
#对在售进行数据处理
df_z=DataFrame(g2,columns=['title','totalPrice','unitPrice','room','type','area','quxian','district','seller','builtYear','guapaiDate','preDate'])
df_z['builtYear']=df_z['builtYear'].map(lambda x:int(re.search('\d+',x).group()) if not '未知' in x else 0)
df_z['area']=df_z['area'].apply(lambda x:float(re.search('\d+(\.\d+)?',x).group()))
df_z=df_z.astype({'totalPrice':'float','unitPrice':'float'})
df_z['guapaiDate']=df_z['guapaiDate'].apply(lambda x:str.replace(x,'-','.'))#要把2021-01-10转换为2021.01.10
df_z['guapaiDate']=pd.to_datetime(df_z['guapaiDate'])
df_z['preDate']=df_z['preDate'].apply(lambda x:str.replace(x,'-','.'))
df_z['preDate']=df_z['preDate'].apply(lambda x:pd.to_datetime(x) if x!='暂无数据' else 0)

In [ ]:
d=round(df_z.groupby(['title'])['unitPrice'].describe(),0)
d.columns.name='正在售'
d

In [ ]:
d1=df_c[(df_c['dealDate'].apply(lambda x:x.year==2020))| (df_c['dealDate'].apply(lambda x:x.year==2021))]
d=round(d1.groupby(['title'])['unitPrice'].describe(),0)
d.columns.name='已成交'
d